## Get lots of csv files of daily depth data and concatenate them into one file of monthly gauge data
This notebook generates a csv file of average monthly depth data for multiple reservoirs. This csv file can be copied and pasted into the depth-to-surface area spreadsheet and index matched so you can get monthly surface area based on monthly depth.  

In [1]:
import os
import pandas as pd
from tqdm.auto import tqdm #this is just the progress bar function that I like

In [2]:
#Define the list of gauge IDs you want to get 
ID_list = ['210117','222538','401027','410543','410748','412106',
           '419041','419069','425022','130314B','138012A','138121A',
           '232217A','422315B','MALMSBURY','ODSS_21166_WSLAHD.1',
           'ROCKLANDS','sp-o10334','sp-o11454','sp-o11534','TAYLORS',
           '412010','222539','629.1','613.1']

test_files_list = []

#Find the files that represent those gauge IDs (thanks to my good naming convention practices, the gauge ID is in the file name)
directory = '00_Library'
for filename in os.listdir(directory):
    for i in ID_list:
        if i in filename:
            test_files_list.append(os.path.join(directory, filename))

data_dict = {}        
ID_list2 = []

#Make a dictionary of pandas dataframes where the key is the gauge ID and make it monthly instead of daily
#The ID has to be read in the same loop as the data to make sure they are the same one
for i in tqdm(test_files_list, total=len(test_files_list)):
    df = pd.read_csv(i, nrows=1, escapechar='#')
    column = df.iloc[:,[1]] #This is the column with the ID in it
    ID = list(column)
    ID = ID[0]
    ID = df.at[0, ID]
    ID_list2.append(str(ID))
    
    data = pd.read_csv(i, error_bad_lines = False, skiprows=9, escapechar='#',
                         parse_dates=['Timestamp'], 
                         index_col=('Timestamp'),
                        date_parser=lambda x: pd.to_datetime(x.rsplit('+', 1)[0]))
    data = data.drop(columns=['Quality Code', 'Interpolation Type'])
    data = data.resample('MS').mean() # resample() is a pandas function and 'MS' stands for monthly summary
    data_dict.update({str(ID): data})
    
data_list = []

#loop over the dictionary to add the ID column
for i in ID_list:
    df = data_dict[i]
    df['ID'] = i
    data_list.append(df)
    
#concatenate the dataframes into one and round values to integers
df = pd.concat(data_list)
df['Value'] = df['Value'].round(0)
df

,Value,ID
Timestamp,,
2000-01-01,180.0,210117
2000-02-01,179.0,210117
2000-03-01,180.0,210117
2000-04-01,182.0,210117
2000-05-01,182.0,210117
...,...,...
2020-07-01,473.0,613.1
2020-08-01,473.0,613.1
2020-09-01,470.0,613.1


Great, now you can save it as a csv file. When it appears in the directory on the side, you can right click and download to your computer.

In [ ]:
df.to_csv('test_data.csv')